In [2]:
import requests
from tqdm.notebook import tqdm as tqdm_nb
import pandas as pd

all_data = []  # Initialize an empty list to store the combined data
for i in tqdm_nb(range(2010, 2020)):
    url = f'https://api.tfl.gov.uk/AccidentStats/{i}'
    response = requests.get(url)
    if response.status_code == 200:
        all_data.extend(response.json())  # Extend the list with data from each year
    else:
        print(f"Error retrieving data for year {i}: {response.status_code}")

# Convert the combined data list into a DataFrame
df_accidents = pd.DataFrame(all_data).drop(columns='$type')

# Display the DataFrame
df_accidents.head(10).style

  0%|          | 0/10 [00:00<?, ?it/s]

,id,lat,lon,location,date,severity,borough,casualties,vehicles
0,121516,51.513694,-0.156189,Portman Street junction with Oxford Street,2010-01-27T01:15:00Z,Serious,City of Westminster,"[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Casualty, Tfl.Api.Presentation.Entities', 'age': 32, 'class': 'Driver', 'severity': 'Serious', 'mode': 'PedalCycle', 'ageBand': 'Adult'}]","[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Vehicle, Tfl.Api.Presentation.Entities', 'type': 'Car'}, {'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Vehicle, Tfl.Api.Presentation.Entities', 'type': 'PedalCycle'}]"
1,121517,51.522079,-0.157580,Marylebone Road junction with Baker Street,2010-01-27T00:15:00Z,Slight,City of Westminster,"[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Casualty, Tfl.Api.Presentation.Entities', 'age': 35, 'class': 'Driver', 'severity': 'Slight', 'mode': 'PoweredTwoWheeler', 'ageBand': 'Adult'}]","[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Vehicle, Tfl.Api.Presentation.Entities', 'type': 'Motorcycle_125_500cc'}]"
2,121518,51.496713,-0.151400,Belgrave Place junction with Eaton Square,2010-01-26T18:28:00Z,Serious,City of Westminster,"[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Casualty, Tfl.Api.Presentation.Entities', 'age': 37, 'class': 'Driver', 'severity': 'Serious', 'mode': 'PoweredTwoWheeler', 'ageBand': 'Adult'}]","[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Vehicle, Tfl.Api.Presentation.Entities', 'type': 'Motorcycle_500cc_Plus'}, {'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Vehicle, Tfl.Api.Presentation.Entities', 'type': 'Car'}]"
3,121519,51.534302,-0.174389,Wellington Road junction with Grove End Road,2010-01-27T00:08:00Z,Slight,City of Westminster,"[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Casualty, Tfl.Api.Presentation.Entities', 'age': 24, 'class': 'Driver', 'severity': 'Slight', 'mode': 'PoweredTwoWheeler', 'ageBand': 'Adult'}]","[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Vehicle, Tfl.Api.Presentation.Entities', 'type': 'Car'}, {'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Vehicle, Tfl.Api.Presentation.Entities', 'type': 'Motorcycle_50_125cc'}]"
4,121520,51.494771,-0.125403,Millbank junction with Horseferry Road,2010-01-06T08:35:00Z,Slight,City of Westminster,"[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Casualty, Tfl.Api.Presentation.Entities', 'age': 34, 'class': 'Driver', 'severity': 'Slight', 'mode': 'PedalCycle', 'ageBand': 'Adult'}]","[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Vehicle, Tfl.Api.Presentation.Entities', 'type': 'PedalCycle'}, {'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Vehicle, Tfl.Api.Presentation.Entities', 'type': 'Car'}]"
5,121521,51.488982,-0.140189,St George's Drive junction with Charlwood Street,2010-01-22T13:57:00Z,Slight,City of Westminster,"[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Casualty, Tfl.Api.Presentation.Entities', 'age': 32, 'class': 'Pedestrian', 'severity': 'Slight', 'mode': 'Pedestrian', 'ageBand': 'Adult'}]","[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Vehicle, Tfl.Api.Presentation.Entities', 'type': 'Taxi'}]"
6,121522,51.511576,-0.119093,Aldwych junction with Strand,2010-01-23T18:10:00Z,Slight,City of Westminster,"[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Casualty, Tfl.Api.Presentation.Entities', 'age': 36, 'class': 'Driver', 'severity': 'Slight', 'mode': 'PoweredTwoWheeler', 'ageBand': 'Adult'}]","[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Vehicle, Tfl.Api.Presentation.Entities', 'type': 'Taxi'}, {'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Vehicle, Tfl.Api.Presentation.Entities', 'type': 'Motorcycle_50_125cc'}]"
7,121523,51.513253,-0.134012,Wardour Street 45 metres north junction with Peter Street,2010-01-19T08:10:00Z,Slight,City of Westminster,"[{'$type': 'Tfl.Api.Presentation.Entities.AccidentStats.Casualty, Tfl.Api.Presentation.Entities', 'age': 22, 'class': 'Driv

In [3]:
fatal_accidents = []
for accident in tqdm_nb(range(len(df_accidents))):
    casualties = df_accidents['casualties'][accident]
    vehicles = df_accidents['vehicles'][accident]
    if any(casualty['severity'] == 'Fatal' for casualty in casualties):
        vehicles_involved = {}
        for vehicle in vehicles:
            if vehicle['type'] not in vehicles_involved:
                vehicles_involved[vehicle['type']] = 1
            else:
                vehicles_involved[vehicle['type']] += 1
        fatalities_by_mode = {}
        for casualty in casualties:
            if casualty['severity'] == 'Fatal':
                if casualty['mode'] not in fatalities_by_mode:
                    fatalities_by_mode[casualty['mode']] = 1
                else:
                    fatalities_by_mode[casualty['mode']] += 1
        fatal_accidents.append({'vehicles_involved': vehicles_involved,
                                'fatalities_by_mode': fatalities_by_mode})
        
fatal_mask = df_accidents['casualties'].apply(lambda casualties: any(casualty['severity'] == 'Fatal' for casualty in casualties))
fatal_accidents_df = df_accidents[fatal_mask].copy().reset_index().drop(columns='index')
fatal_accidents_df.head(10)

  0%|          | 0/500046 [00:00<?, ?it/s]

,id,lat,lon,location,date,severity,borough,casualties,vehicles
0,131894,51.488749,-0.165406,King's Road 26 metres north east of junction w...,2010-06-23T11:10:00Z,Fatal,Kensington and Chelsea,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
1,126772,51.446035,0.030900,Sidcup Road 95 metres north of junction with M...,2010-04-17T22:00:00Z,Fatal,Greenwich,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
2,126775,51.480842,-0.021993,Creek Road 50 metres east of junction with Dep...,2010-05-25T17:47:00Z,Fatal,Greenwich,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
3,126786,51.477258,0.019470,Charlton Road junction with Old Dover Road,2010-09-23T12:35:00Z,Fatal,Greenwich,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
4,126796,51.443323,0.052366,Court Road 80 metres north junction with Tarnw...,2010-01-16T07:05:00Z,Fatal,Greenwich,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
5,126803,51.441424,0.073433,Tiverton Drive 59 metres north east of junctio...,2010-03-21T14:43:00Z,Fatal,Greenwich,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
6,131905,51.480158,-0.179576,Cremorne Road 35 metres east of junction with ...,2010-12-28T12:03:00Z,Fatal,Kensington and Chelsea,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
7,131914,51.529462,-0.222303,Harrow Road 98 metres south east of junction w...,2010-10-11T00:55:00Z,Fatal,Kensington and Chelsea,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
8,129657,51.480387,-0.108855,Camberwell New Road 28 metres north west of ju...,2010-10-07T17:29:00Z,Fatal,Lambeth,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
9,129688,51.444643,-0.139833,Cambray Road 134 metres east of junction with ...,2010-03-28T13:52:00Z,Fatal,Lambeth,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...


In [5]:
import pandas as pd
from weather_scrape import fetch_weather_data

# Drop columns containing lists
columns_to_drop = ['casualties', 'vehicles']
fatal_accidents_df = fatal_accidents_df.drop(columns=columns_to_drop, errors='ignore')

# Remove duplicates from the DataFrame
fatal_accidents_df = fatal_accidents_df.drop_duplicates()

# Ensure 'date' column is in datetime format
fatal_accidents_df['date'] = pd.to_datetime(fatal_accidents_df['date'])

# Create a new column for truncated datetime to the hour level
fatal_accidents_df['date_hour'] = fatal_accidents_df['date'].dt.floor('H')

# Initialize an empty list to store weather data
weather_data_list = []

# Iterate over each accident
for index, row in fatal_accidents_df.iterrows():
    latitude = row['lat']
    longitude = row['lon']
    timestamp_hour = row['date_hour']

    try:
        weather_data = fetch_weather_data(latitude, longitude, timestamp_hour)
        weather_data_dict = weather_data.to_dict()  # Convert the row to a dictionary
        weather_data_dict.update({
            'lat': latitude,
            'lon': longitude,
            'date_hour': timestamp_hour  # Use the truncated datetime for consistency
        })
        weather_data_list.append(weather_data_dict)
    except Exception as e:
        print(f"Error fetching weather data for {timestamp_hour} at ({latitude}, {longitude}): {e}")

# Convert the list of weather data to a DataFrame
weather_data_df = pd.DataFrame(weather_data_list)


# Ensure 'date_hour' column is in datetime format in the weather data DataFrame
weather_data_df['date_hour'] = pd.to_datetime(weather_data_df['date_hour'])


# Merge the weather data with the fatal accidents data on 'lat', 'lon', and 'date_hour'
merged_df = pd.merge(fatal_accidents_df, weather_data_df, on=['lat', 'lon', 'date_hour'])

# Drop the 'date_hour' column from the merged DataFrame if no longer needed
merged_df = merged_df.drop(columns=['date_hour'], errors='ignore')

# Remove any potential duplicates after merge
merged_df = merged_df.drop_duplicates()

# Save or use the merged dataframe for further analysis
merged_df.to_csv('merged_fatal_accidents_weather.csv', index=False)

print("Merged data saved to 'merged_fatal_accidents_weather.csv'")


/var/folders/l8/6hvdql9j5f56gk4szb6ryywr0000gn/T/ipykernel_88475/367765806.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  fatal_accidents_df['date_hour'] = fatal_accidents_df['date'].dt.floor('H')


Merged data saved to 'merged_fatal_accidents_weather.csv'


In [17]:
# Print the number of rows in the merged DataFrame
print("Number of rows in merged_df:", merged_df.shape[0])

Number of rows in merged_df: 2540


In [14]:
fatal_accidents_df.shape

(2540, 7)

In [18]:
import time
from weather_scrape import fetch_weather_data

# Drop columns containing lists
columns_to_drop = ['casualties', 'vehicles']
df_accidents = df_accidents.drop(columns=columns_to_drop, errors='ignore')

# Remove duplicates from the DataFrame
df_accidents = df_accidents.drop_duplicates()

# Initialize an empty list to store weather data
weather_data_list = []

# Define batch size and delay
batch_size = 8000  # Adjust batch size as needed
delay_seconds = 90  # Delay between batches in seconds

# Iterate over each accident in batches
for start_index in range(0, len(df_accidents), batch_size):
    batch = df_accidents.iloc[start_index:start_index + batch_size]
    for index, row in batch.iterrows():
        latitude = row['lat']
        longitude = row['lon']
        timestamp = row['date']  # Assuming 'datetime' column contains the datetime information
        # Convert 'timestamp' to datetime object if it's not already
        if not isinstance(timestamp, pd.Timestamp):
            timestamp = pd.to_datetime(timestamp)
        
        # Truncate the timestamp to the hour level
        timestamp_hour = timestamp.replace(minute=0, second=0, microsecond=0)

        try:
            weather_data = fetch_weather_data(latitude, longitude, timestamp_hour)
            weather_data_dict = weather_data.to_dict()  # Convert the row to a dictionary
            weather_data_dict.update({
                'lat': latitude,
                'lon': longitude,
                'datetime': timestamp,  # Retain the original datetime
            })
            weather_data_list.append(weather_data_dict)
        except Exception as e:
            print(f"Error fetching weather data for {timestamp_hour} at ({latitude}, {longitude}): {e}")

    # Introduce a delay between batches
    print(f"Processed batch {start_index // batch_size + 1}, sleeping for {delay_seconds} seconds...")
    time.sleep(delay_seconds)

# Convert the list of weather data to a DataFrame
weather_data_df = pd.DataFrame(weather_data_list)

# Merge the weather data with the accident data
merged_full_df = pd.merge(df_accidents, weather_data_df, on=['lat', 'lon', 'date'])

# Drop duplicates from the merged DataFrame
merged_full_df = merged_full_df.drop_duplicates()

# Save or use the merged dataframe for further analysis
merged_full_df.to_csv('merged_all_accidents_weather.csv', index=False)


Processed batch 1, sleeping for 90 seconds...
Error fetching weather data for 2010-04-14 12:00:00+00:00 at (51.513282, 0.000598): {'error': True, 'reason': 'Minutely API request limit exceeded. Please try again in one minute.'}
Error fetching weather data for 2010-03-19 09:00:00+00:00 at (51.507365, -0.016348): {'reason': 'Minutely API request limit exceeded. Please try again in one minute.', 'error': True}
Error fetching weather data for 2010-01-31 17:00:00+00:00 at (51.509639, -0.055476): {'error': True, 'reason': 'Minutely API request limit exceeded. Please try again in one minute.'}
Error fetching weather data for 2010-04-23 17:00:00+00:00 at (51.517405, -0.073739): {'reason': 'Minutely API request limit exceeded. Please try again in one minute.', 'error': True}
Error fetching weather data for 2010-04-22 08:00:00+00:00 at (51.514101, -0.053413): {'error': True, 'reason': 'Minutely API request limit exceeded. Please try again in one minute.'}
Error fetching weather data for 2010-04-

KeyboardInterrupt: 

In [19]:
import time
import pandas as pd
from weather_scrape import fetch_weather_data

# Drop columns containing lists
columns_to_drop = ['casualties', 'vehicles']
df_accidents = df_accidents.drop(columns=columns_to_drop, errors='ignore')

# Remove duplicates from the DataFrame
df_accidents = df_accidents.drop_duplicates()

# Initialize an empty list to store weather data
weather_data_list = []

# Define batch size and delay
batch_size = 8000  # Adjust batch size as needed
delay_seconds = 90  # Delay between batches in seconds

# Process only the first batch
start_index = 0
batch = df_accidents.iloc[start_index:start_index + batch_size]

for index, row in batch.iterrows():
    latitude = row['lat']
    longitude = row['lon']
    timestamp = row['date']  # Assuming 'date' column contains the datetime information
    # Convert 'timestamp' to datetime object if it's not already
    if not isinstance(timestamp, pd.Timestamp):
        timestamp = pd.to_datetime(timestamp)
    
    # Truncate the timestamp to the hour level
    timestamp_hour = timestamp.replace(minute=0, second=0, microsecond=0)

    try:
        weather_data = fetch_weather_data(latitude, longitude, timestamp_hour)
        weather_data_dict = weather_data.to_dict()  # Convert the row to a dictionary
        weather_data_dict.update({
            'lat': latitude,
            'lon': longitude,
            'date': timestamp,  # Retain the original datetime
        })
        weather_data_list.append(weather_data_dict)
    except Exception as e:
        print(f"Error fetching weather data for {timestamp_hour} at ({latitude}, {longitude}): {e}")

# Convert the list of weather data to a DataFrame
weather_data_df = pd.DataFrame(weather_data_list)

# Ensure 'date' column is in datetime format in both DataFrames
df_accidents['date'] = pd.to_datetime(df_accidents['date'])
weather_data_df['date'] = pd.to_datetime(weather_data_df['date'])

# Merge the weather data with the accident data
merged_df = pd.merge(df_accidents, weather_data_df, on=['lat', 'lon', 'date'])

# Drop duplicates from the merged DataFrame
merged_df = merged_df.drop_duplicates()

# Save the merged dataframe of the first batch to a CSV file
merged_df.to_csv('merged_first_batch_accidents_weather.csv', index=False)

print("Merged first batch saved to 'merged_first_batch_accidents_weather.csv'")

Merged first batch saved to 'merged_first_batch_accidents_weather.csv'


In [23]:
import time
import pandas as pd
from weather_scrape import fetch_weather_data
import requests

# Function to check and handle API rate limiting
def check_rate_limit(response):
    headers = response.headers
    if 'X-RateLimit-Remaining' in headers and 'X-RateLimit-Reset' in headers:
        remaining = int(headers['X-RateLimit-Remaining'])
        reset_time = int(headers['X-RateLimit-Reset'])
        current_time = int(time.time())
        sleep_time = max(0, reset_time - current_time)
        return remaining, sleep_time
    else:
        return None, None

# Function to fetch weather data with retry and rate limit handling
def fetch_weather_data_with_limit_handling(latitude, longitude, timestamp_hour):
    for attempt in range(3):  # Retry up to 3 times
        try:
            response = fetch_weather_data(latitude, longitude, timestamp_hour)
            response.raise_for_status()  # Raise an HTTPError if the HTTP request returned an unsuccessful status code
            remaining_requests, sleep_time = check_rate_limit(response)
            if remaining_requests is not None and remaining_requests == 0:
                print(f"Rate limit exceeded. Sleeping for {sleep_time} seconds.")
                time.sleep(sleep_time + 1)  # Adding 1 second to ensure the limit has reset
            return response.json()  # Ensure we're returning the JSON content of the response
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < 2:  # Don't wait after the last attempt
                time.sleep(5)  # Wait a bit before retrying
    raise Exception("Failed to fetch weather data after 3 attempts")

# Drop columns containing lists
columns_to_drop = ['casualties', 'vehicles']
df_accidents = df_accidents.drop(columns=columns_to_drop, errors='ignore')

# Remove duplicates from the DataFrame
df_accidents = df_accidents.drop_duplicates()

# Initialize an empty list to store weather data
weather_data_list = []

# Start processing the data
batch_size = 8000  # Adjust batch size as needed
delay_seconds = 90  # Delay between batches in seconds
total_requests = 0

# Iterate over each accident in batches
for start_index in range(0, len(df_accidents), batch_size):
    batch = df_accidents.iloc[start_index:start_index + batch_size]
    for index, row in batch.iterrows():
        latitude = row['lat']
        longitude = row['lon']
        timestamp = row['date']  # Assuming 'date' column contains the datetime information
        # Convert 'timestamp' to datetime object if it's not already
        if not isinstance(timestamp, pd.Timestamp):
            timestamp = pd.to_datetime(timestamp)
        
        # Truncate the timestamp to the hour level
        timestamp_hour = timestamp.replace(minute=0, second=0, microsecond=0)

        try:
            weather_data = fetch_weather_data_with_limit_handling(latitude, longitude, timestamp_hour)
            weather_data.update({
                'lat': latitude,
                'lon': longitude,
                'date': timestamp,  # Retain the original datetime
            })
            weather_data_list.append(weather_data)
            total_requests += 1

        except Exception as e:
            print(f"Error fetching weather data for {timestamp_hour} at ({latitude}, {longitude}): {e}")

    # Save intermediate results
    weather_data_df = pd.DataFrame(weather_data_list)
    weather_data_df['date'] = pd.to_datetime(weather_data_df['date'])
    merged_df = pd.merge(df_accidents, weather_data_df, on=['lat', 'lon', 'date'])
    merged_df = merged_df.drop_duplicates()
    merged_df.to_csv(f'merged_intermediate_{start_index // batch_size + 1}_accidents_weather.csv', index=False)
    print(f"Merged intermediate batch saved to 'merged_intermediate_{start_index // batch_size + 1}_accidents_weather.csv'")
    weather_data_list = []  # Reset for the next batch

    # Introduce a delay between batches
    print(f"Processed batch {start_index // batch_size + 1}, sleeping for {delay_seconds} seconds...")
    time.sleep(delay_seconds)

# Final save for any remaining data
if weather_data_list:
    weather_data_df = pd.DataFrame(weather_data_list)
    weather_data_df['date'] = pd.to_datetime(weather_data_df['date'])
    merged_df = pd.merge(df_accidents, weather_data_df, on=['lat', 'lon', 'date'])
    merged_df = merged_df.drop_duplicates()
    merged_df.to_csv('merged_all_accidents_weather.csv', index=False)
    print("Merged final data saved to 'merged_all_accidents_weather.csv'")


Error fetching weather data for 2010-01-27 01:00:00+00:00 at (51.513694, -0.156189): 'Series' object has no attribute 'raise_for_status'
Error fetching weather data for 2010-01-27 00:00:00+00:00 at (51.522079, -0.15758): 'Series' object has no attribute 'raise_for_status'
Error fetching weather data for 2010-01-26 18:00:00+00:00 at (51.496713, -0.1514): 'Series' object has no attribute 'raise_for_status'
Error fetching weather data for 2010-01-27 00:00:00+00:00 at (51.534302, -0.174389): 'Series' object has no attribute 'raise_for_status'
Error fetching weather data for 2010-01-06 08:00:00+00:00 at (51.494771, -0.125403): 'Series' object has no attribute 'raise_for_status'
Error fetching weather data for 2010-01-22 13:00:00+00:00 at (51.488982, -0.140189): 'Series' object has no attribute 'raise_for_status'
Error fetching weather data for 2010-01-23 18:00:00+00:00 at (51.511576, -0.119093): 'Series' object has no attribute 'raise_for_status'
Error fetching weather data for 2010-01-19 0

KeyError: 'date'